## Sorter Song

In [8]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

In [9]:
%run "../setup/Setup_Generic.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [10]:
cw.program_target(scope, prog, "sortersSong-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 14551 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 14551 bytes


In [11]:
import chipwhisperer as cw
import numpy as np
import time
import struct
from sklearn.linear_model import LogisticRegression

########################################
# ChipWhisperer setup
########################################

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)

########################################
# Globals / cache / ML storage
########################################

cache = {}  # (value, position, i) -> trace fragment

# Training data for branch decisions:
# X = features extracted from (trace_low, trace_mid, trace_high)
# y = 1 => go LEFT (keep [low..mid]); 0 => go RIGHT (keep [mid..high])
ml_data_X = []
ml_data_y = []

ml_model = None
ml_trained = False
_seen_classes = set()

# Training controls
MIN_SAMPLES_TO_TRAIN = 20
RETRAIN_EVERY_STEPS = 10   # try retraining every N branch steps
DEBUG = True               # set False to silence per-step path logs


########################################
# Basic target interaction helpers
########################################

def num_q():
    target.simpleserial_write('q', b"0")
    response = target.simpleserial_read('r', 4)
    return struct.unpack("<I", response)[0]

def reset():
    target.simpleserial_write('x', b"")
    response = target.simpleserial_read('r', 1)
    return response

def get_pt(pt):
    target.simpleserial_write('p', bytes(pt))
    response = target.simpleserial_read('r', 2)
    return response

def sort_data(i):
    # capture a trace after running the sort/compare routine
    scope.arm()
    cmd = 'c' if i == 1 else 'd'
    target.simpleserial_write(cmd, b"")
    timeout = scope.capture()
    if timeout:
        raise RuntimeError("Timeout")
    response = target.simpleserial_read('r', 1)
    trace = scope.get_last_trace()
    return trace

def check_array(arr, i):
    # send recovered array back to device for validation / flag
    cmd = 'a' if i == 1 else 'b'
    bytes_arr = []
    for val in arr:
        bytes_arr.append(val & 0xFF)
        if i == 2:
            bytes_arr.append((val >> 8) & 0xFF)
    target.simpleserial_write(cmd, bytes(bytes_arr))
    response = target.simpleserial_read('r', 20)
    return response

########################################
# Trace acquisition + caching 
########################################

def get_trace(value, position, i):
    """
    Prepare target state for a given guess 'value' at array index 'position'
    for array type i (1 = 8-bit arr, 2 = 16-bit arr).
    Returns the first 700 samples of the captured power trace.

    We memoize by (value, position, i) so repeated queries are cheap.
    """
    key = (value, position, i)
    if key in cache:
        return cache[key]
    else:
        low = value & 0xFF
        high = (value >> 8) & 0xFF
        pt = [i, low, high, position]

        reset()
        get_pt(pt)
        trace = sort_data(i)
        t = trace[:700]
        cache[key] = t
        return t

########################################
# Feature extraction for ML
########################################

def extract_features(trace_low, trace_mid, trace_high):
    """
    Turn (trace_low, trace_mid, trace_high) into a numeric feature vector.

    We include:
    - L1 distances between pairs
    - L2 distances between pairs
    - A few simple aggregate stats of each trace
    """
    tl = np.array(trace_low, dtype=np.float32)
    tm = np.array(trace_mid, dtype=np.float32)
    th = np.array(trace_high, dtype=np.float32)

    # pairwise diffs
    diff_lm_abs = np.sum(np.abs(tm - tl))
    diff_mh_abs = np.sum(np.abs(th - tm))
    diff_lh_abs = np.sum(np.abs(th - tl))

    diff_lm_l2 = np.sqrt(np.sum((tm - tl) ** 2))
    diff_mh_l2 = np.sqrt(np.sum((th - tm) ** 2))
    diff_lh_l2 = np.sqrt(np.sum((th - tl) ** 2))

    # simple stats per trace
    feats = [
        diff_lm_abs, diff_mh_abs, diff_lh_abs,
        diff_lm_l2, diff_mh_l2, diff_lh_l2,
        float(np.mean(tl)), float(np.mean(tm)), float(np.mean(th)),
        float(np.std(tl)),  float(np.std(tm)),  float(np.std(th)),
        float(np.max(tl) - np.min(tl)),
        float(np.max(tm) - np.min(tm)),
        float(np.max(th) - np.min(th)),
    ]
    return np.array(feats, dtype=np.float32)

########################################
# ML training / prediction helpers
########################################

_total_steps = 0  # global branch step counter (for periodic retraining)

def maybe_train_model():
    """
    Train/retrain logistic regression when enough diverse samples exist.
    Prints when the model is (re)trained.
    """
    global ml_model, ml_trained

    if len(ml_data_X) < MIN_SAMPLES_TO_TRAIN:
        return

    X = np.array(ml_data_X, dtype=np.float32)
    y = np.array(ml_data_y, dtype=np.int32)

    # need both classes (LEFT and RIGHT) to train a classifier
    if len(np.unique(y)) < 2:
        return

    clf = LogisticRegression(max_iter=200)
    clf.fit(X, y)
    ml_model = clf

    if not ml_trained and DEBUG:
        print(f"[ML] trained first model on {len(X)} steps")
    elif DEBUG:
        print(f"[ML] retrained model on {len(X)} steps")

    ml_trained = True


def model_decision(features, diff_low_mid, diff_mid_high):
    """
    Decide which side of mid the true secret lies on.
    Returns (go_left, used_ml):
       go_left True  -> keep [low .. mid]  => high = mid
       go_left False -> keep [mid .. high] => low  = mid
       used_ml True if ML made the call, else False (heuristic)
    """
    # Heuristic: if diff(mid,low) > diff(high,mid) --> go LEFT
    heuristic_left = (diff_low_mid > diff_mid_high)

    if ml_trained and ml_model is not None:
        f = features.reshape(1, -1)
        pred = ml_model.predict(f)[0]   # 1 => left, 0 => right
        return bool(pred), True
    else:
        return heuristic_left, False


def record_training_sample(features, chose_left):
    """
    Store (features -> decision) as a labeled training sample.
    """
    ml_data_X.append(features)
    label = 1 if chose_left else 0
    ml_data_y.append(label)
    _seen_classes.add(label)

########################################
# Binary search with ML-guided branching (with path logging)
########################################

def binary_search(position, high, i):
    """
    Recover the secret value at array index 'position' for array type i,
    assuming the array is strictly decreasing (so we can shrink 'high').

    For each step:
      - capture traces for low, mid, high (unchanged I/O)
      - compute features, diffs
      - decide branch via ML if trained, else heuristic
      - print a one-line debug showing which path and whether ML was used
      - log the decision as training data and retrain periodically
    """
    global _total_steps

    low = 0

    while (high - low) > 1:
        mid = (low + high) // 2

        trace_low  = get_trace(low,  position, i)
        trace_mid  = get_trace(mid,  position, i)
        trace_high = get_trace(high, position, i)

        # diffs from your original rule
        diff_low_mid  = float(np.sum(np.abs(np.array(trace_mid)  - np.array(trace_low))))
        diff_mid_high = float(np.sum(np.abs(np.array(trace_high) - np.array(trace_mid))))

        # ML features
        feats = extract_features(trace_low, trace_mid, trace_high)

        # Decide branch
        go_left, used_ml = model_decision(feats, diff_low_mid, diff_mid_high)

        # Path logging (before updating bounds, so the printed range corresponds to the decision point)
        if DEBUG:
            print(
                f"[PATH] pos={position} i={i} range=[{low},{high}] mid={mid} "
                f"-> {'LEFT' if go_left else 'RIGHT'} "
                f"({'ML' if used_ml else 'heur'}) "
                f"d_lm={diff_low_mid:.2f} d_mh={diff_mid_high:.2f}"
            )

        # Apply branch
        if go_left:
            high = mid
        else:
            low = mid

        # Log + maybe retrain
        record_training_sample(feats, go_left)
        _total_steps += 1
        if _total_steps % RETRAIN_EVERY_STEPS == 0:
            maybe_train_model()

    return low

########################################
# High-level solve routine
########################################

def solve_array(i):
    """
    Solve array #i (1 = 8-bit elements, 2 = 16-bit elements).
    Uses binary_search() for each index from 14 down to 0,
    respecting the monotonic decreasing constraint by shrinking 'high'.
    """
    print(f"Solving for array {i}")
    recovered_array = []
    queries_before = num_q()

    # high bound depends on element width
    high = (2**8  - 1) if i == 1 else (2**16 - 1)

    # The challenge logic: values are strictly decreasing,
    # so once we find one index's value, we update 'high' to that-1.
    for position in [idx for idx in range(14, -1, -1)]:
        value = binary_search(position, high, i)
        recovered_array.append(value)
        high = value - 1

    # reverse because we solved from last index down to first
    recovered_array = recovered_array[::-1]

    response = check_array(recovered_array, i)
    if response:
        decoded = response.decode('utf-8', errors='ignore')
        print(f"Flag (array {i}): {decoded}")
        print(f"Recovered array {i}: {recovered_array}")

    queries_after = num_q()
    print(f"Total Queries for array {i}: {queries_after - queries_before}\n")


(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


## Sorters Song 1

In [6]:
########################################
# Main
########################################

def main():
    scope.default_setup()

    solve_array(1)

    # Cleanup
    scope.dis()
    target.dis()

if __name__ == "__main__":
    main()


Solving for array 1
[PATH] pos=14 i=1 range=[0,255] mid=127 -> RIGHT (heur) d_lm=3.68 d_mh=127.25
[PATH] pos=14 i=1 range=[127,255] mid=191 -> RIGHT (heur) d_lm=3.32 d_mh=127.35
[PATH] pos=14 i=1 range=[191,255] mid=223 -> LEFT (heur) d_lm=127.39 d_mh=3.89
[PATH] pos=14 i=1 range=[191,223] mid=207 -> RIGHT (heur) d_lm=3.24 d_mh=127.21
[PATH] pos=14 i=1 range=[207,223] mid=215 -> RIGHT (heur) d_lm=2.98 d_mh=127.25
[PATH] pos=14 i=1 range=[215,223] mid=219 -> RIGHT (heur) d_lm=3.79 d_mh=127.30
[PATH] pos=14 i=1 range=[219,223] mid=221 -> LEFT (heur) d_lm=127.41 d_mh=2.94
[PATH] pos=14 i=1 range=[219,221] mid=220 -> LEFT (heur) d_lm=126.73 d_mh=3.56
[PATH] pos=13 i=1 range=[0,218] mid=109 -> RIGHT (heur) d_lm=3.41 d_mh=125.50
[PATH] pos=13 i=1 range=[109,218] mid=163 -> RIGHT (heur) d_lm=3.60 d_mh=125.71
[PATH] pos=13 i=1 range=[163,218] mid=190 -> LEFT (heur) d_lm=125.48 d_mh=2.46
[PATH] pos=13 i=1 range=[163,190] mid=176 -> RIGHT (heur) d_lm=4.05 d_mh=125.54
[PATH] pos=13 i=1 range=[176

## Sorters Song 2

In [12]:
########################################
# Main
########################################

def main():
    scope.default_setup()

    solve_array(2)

    # Cleanup
    scope.dis()
    target.dis()

if __name__ == "__main__":
    main()


Solving for array 2
[PATH] pos=14 i=2 range=[0,65535] mid=32767 -> RIGHT (heur) d_lm=4.19 d_mh=99.20
[PATH] pos=14 i=2 range=[32767,65535] mid=49151 -> RIGHT (heur) d_lm=3.89 d_mh=99.30
[PATH] pos=14 i=2 range=[49151,65535] mid=57343 -> LEFT (heur) d_lm=98.95 d_mh=3.29
[PATH] pos=14 i=2 range=[49151,57343] mid=53247 -> RIGHT (heur) d_lm=3.13 d_mh=98.65
[PATH] pos=14 i=2 range=[53247,57343] mid=55295 -> LEFT (heur) d_lm=98.71 d_mh=3.34
[PATH] pos=14 i=2 range=[53247,55295] mid=54271 -> RIGHT (heur) d_lm=3.69 d_mh=98.53
[PATH] pos=14 i=2 range=[54271,55295] mid=54783 -> LEFT (heur) d_lm=98.23 d_mh=3.90
[PATH] pos=14 i=2 range=[54271,54783] mid=54527 -> LEFT (heur) d_lm=97.80 d_mh=3.14
[PATH] pos=14 i=2 range=[54271,54527] mid=54399 -> RIGHT (heur) d_lm=2.99 d_mh=98.02
[PATH] pos=14 i=2 range=[54399,54527] mid=54463 -> RIGHT (heur) d_lm=3.11 d_mh=97.71
[PATH] pos=14 i=2 range=[54463,54527] mid=54495 -> LEFT (heur) d_lm=97.94 d_mh=3.46
[PATH] pos=14 i=2 range=[54463,54495] mid=54479 -> RIG